In [ ]:
import os
import glob
import numpy as np
import cftime
import h5py
import xesmf as xe   # ya no se usa, pero puedes dejarlo si lo necesitas más adelante
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pyproj import Geod
from matplotlib.colors import LogNorm
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import json
import math

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", module="xarray.coding.times")
warnings.filterwarnings("ignore", module="xarray.coding.cftime_offsets")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="xarray")
warnings.filterwarnings("ignore", module="xarray")

import xarray as xr

# CONFIGURACIÓN GENERAL DE RUTAS
# Define el escenario
escenario = "SSP585"  # Cambia a "SSP585" cuando lo necesites

# Carpeta general donde guardar los resultados
BASE_RESULTADOS = "C:/Users/jaime/Desktop/Universidad/TFG/Resultados"

# Subcarpeta específica del escenario (para gráficos, mapas, etc.)
base_out = os.path.join(BASE_RESULTADOS, escenario, "CAT index")
os.makedirs(base_out, exist_ok=True)

# Archivo común de umbrales (solo se calcula una vez desde SSP245)
UMBRAL_FILE = os.path.join(BASE_RESULTADOS, "umbrales_fijos_desde_ssp245.json")

base_out2 = f"C:/Users/jaime/Desktop/Universidad/TFG/Resultados/{escenario}/zonal wind"
os.makedirs(base_out2, exist_ok=True)

# Lista de modelos a analizar
modelos = [
    "IPSL-CM6A-LR", "HadGEM3-GC31-LL", "CESM2-WACCM", "GFDL-CM4",
    "MPI-ESM1-2-LR", "NorESM2-LM", "ACCESS-CM2",
    "EC-Earth3-Veg-LR", "MIROC6", "TaiESM1"
]

# CARGAR VARIABLES PROCESADAS DESDE LOS .NC
# Ruta donde ya tienes las variables procesadas y guardadas (ua, va, zg, ta)
ruta_guardado = f"D:/TFG/Variables tratadas/{escenario}"

# FUNCIONES ADICIONALES
def asignar_invierno(ds):
    """
    Asigna cada fecha DJF al 'winter_year' correspondiente
    (p. ej., DJF 2040-2041 → invierno 2041)
    """
    meses = ds['time'].dt.month
    años = ds['time'].dt.year
    winter_year = xr.where(meses == 12, años + 1, años)
    return ds.assign_coords(winter_year=("time", winter_year.data))


# FUNCIONES PARA DEF (para TI1)
def _metric_derivs(u, v):
    """
    Calcula derivadas horizontales de u y v sobre una esfera (m/s por metro)
    para estimar el tensor de deformación (stretching y shearing).
    """
    R = 6_371_000.0
    deg2rad = np.pi / 180.0
    rad2deg = 180.0 / np.pi

    lat_rad = np.deg2rad(u['lat'])
    coslat  = np.cos(lat_rad)

    # Derivadas por grado (lo hace xarray)
    dudlon_deg = u.differentiate('lon')   # du/dλ (por grado)
    dudlat_deg = u.differentiate('lat')   # du/dφ (por grado)
    dvdlon_deg = v.differentiate('lon')
    dvdlat_deg = v.differentiate('lat')

    # Convertir "por grado" -> "por radian"
    ## Son derivadas por lo que tengo m/s/grados y quiero (m/s)/rad; por eso se multiplica por 180/pi
    dudlon_rad = dudlon_deg * rad2deg
    dudlat_rad = dudlat_deg * rad2deg
    dvdlon_rad = dvdlon_deg * rad2deg
    dvdlat_rad = dvdlat_deg * rad2deg

    # Pasar a derivadas espaciales (por metro)
    # ∂/∂x = (1/(R cosφ)) ∂/∂λ(rad)
    # ∂/∂y = (1/R) ∂/∂φ(rad)
    dudx = dudlon_rad / (R * coslat)
    dudy = dudlat_rad / R
    dvdx = dvdlon_rad / (R * coslat)
    dvdy = dvdlat_rad / R

    return dudx, dudy, dvdx, dvdy


def compute_def(u, v):
    """
    Calcula el módulo de deformación:
    √[(∂u/∂x - ∂v/∂y)² + (∂v/∂x + ∂u/∂y)²]
    """
    dudx, dudy, dvdx, dvdy = _metric_derivs(u, v)
    stretching = dudx - dvdy
    shearing   = dvdx + dudy
    return np.sqrt(stretching**2 + shearing**2)


# FUNCIONES PARA EL NÚMERO DE RICHARDSON GRADIENTE

def compute_theta(T, p):
    """Temperatura potencial θ [K]"""
    R_cp = 0.2856  # Aproximación R/Cp
    p0 = 100000.0  # Pa
    return T * (p0 / p)**R_cp


def compute_N2(theta, zg):
    """Frecuencia de Brunt–Väisälä al cuadrado (N²)"""
    g = 9.80665
    dtheta_dplev = theta.differentiate("plev")
    dz_dplev = zg.differentiate("plev")
    dtheta_dz = dtheta_dplev / dz_dplev
    N2 = (g / theta) * dtheta_dz
    # Quita infinitos/nans raros
    return N2.where(np.isfinite(N2))


def compute_shear2(u, v, zg):
    """Magnitud del shear vertical²"""
    dudz = u.differentiate("plev") / zg.differentiate("plev")
    dvdz = v.differentiate("plev") / zg.differentiate("plev")
    shear2 = dudz**2 + dvdz**2
    # Evita valores ~0 que disparan Ri
    return shear2.where(shear2 > 1e-8)  # s^-2 (umbral conservador)


def compute_Ri(u, v, T, zg):
    """
    Calcula el número de Richardson Ri = N² / (∂V/∂z)²,
    filtrando solo zonas de estratificación estable (N² > 0)
    """
    theta = compute_theta(T, u["plev"])
    N2 = compute_N2(theta, zg)
    shear2 = compute_shear2(u, v, zg)
    N2 = N2.where(N2 > 0)
    Ri = N2 / shear2
    return Ri

# EJEMPLO RUTA TRANSATLÁNTICA
lon_DUB, lat_DUB = -6.2621, 53.4287   # Dublín
lon_JFK, lat_JFK = -73.7781, 40.6413  # Nueva York JFK
geod = Geod(ellps="WGS84")

In [ ]:
# REINICIO DEL SCRIPT DESDE VARIABLES YA PROCESADAS

ua_models, va_models, zg_models, ta_models = [], [], [], []
shear_slopes = []
VWS_mean_models = []
ti1_slopes = []
ti1_mean_models = []
ti1_signif_models = []
ti1_daily_all_models = []
VWS_daily_all_models = []
Ri_slopes = []

for modelo in modelos:
    ruta_modelo = os.path.join(ruta_guardado, modelo)
    if not os.path.isdir(ruta_modelo):
        print(f" {modelo}: carpeta no encontrada ({ruta_modelo})")
        continue

    try:
        ua = xr.open_dataset(os.path.join(ruta_modelo, f"{modelo}_ua_DJF.nc"))["ua"]
        va = xr.open_dataset(os.path.join(ruta_modelo, f"{modelo}_va_DJF.nc"))["va"]
        zg = xr.open_dataset(os.path.join(ruta_modelo, f"{modelo}_zg_DJF.nc"))["zg"]
        ta = xr.open_dataset(os.path.join(ruta_modelo, f"{modelo}_ta_DJF.nc"))["ta"]
    except Exception as e:
        print(f"{modelo}: error al cargar archivos procesados ({e})")
        continue

    # Añadir coordenada 'modelo' para mantener compatibilidad
    ua = ua.expand_dims(modelo=[modelo])
    va = va.expand_dims(modelo=[modelo])
    zg = zg.expand_dims(modelo=[modelo])
    ta = ta.expand_dims(modelo=[modelo])

    ua_models.append(ua)
    va_models.append(va)
    zg_models.append(zg)
    ta_models.append(ta)

    print(f"{modelo}: variables procesadas cargadas correctamente.")

    print("\n Todas las variables procesadas han sido cargadas.")
    print(f"Modelos cargados: {[m.modelo.values.item() for m in ua_models]}")

    # CÁLCULOS DE VWS USANDO zg
    u250 = ua.sel(plev=25000, method='nearest')
    u500 = ua.sel(plev=50000, method='nearest')
    v250 = va.sel(plev=25000, method='nearest')
    v500 = va.sel(plev=50000, method='nearest')
    dz = zg.sel(plev=25000, method='nearest') - zg.sel(plev=50000, method='nearest')
    if zg.plev.values[0] < zg.plev.values[-1]:
        dz = -dz  # corrige el signo si necesario

    # Evitar divisiones por cero o por capas demasiado delgadas
    dz = dz.where((dz > 100) & (dz < 10000))
    VWS = np.sqrt((u250 - u500)**2 + (v250 - v500)**2) / dz

    # GUARDAR VWS DIARIO DJF (para percentiles multimodelo)
    VWS = VWS.where(np.isfinite(VWS))  # elimina inf o valores absurdos
    VWS = VWS.rename("VWS")
    VWS_daily_all_models.append(VWS) 

    VWS_with_year = asignar_invierno(VWS)
    VWS_djf_yearly = VWS_with_year.groupby("winter_year").mean("time")

    # Media climatológica DJF del TI1 (2030–2100)
    VWS_mean = VWS_djf_yearly.mean("winter_year")
    VWS_mean = VWS_mean.rename("VWS_mean")

    # Guardar para multimodelo
    # VWS_mean = VWS_mean.assign_coords(modelo=modelo)
    VWS_mean_models.append(VWS_mean)

    ######################################################################################
    VWS_trend = VWS_djf_yearly.polyfit(dim="winter_year", deg=1)
    VWS_slope = VWS_trend["polyfit_coefficients"].sel(degree=1) * 10
    # VWS_slope = VWS_slope.assign_coords(modelo=modelo)
    shear_slopes.append(VWS_slope)

    print(f"\n {modelo}: diagnóstico VWS/TI1")
    print(f" - zg units: {zg.attrs.get('units', 'sin unidades')}")
    print(f" - plev: {ua.plev.values}")
    print(f" - zg500 media: {float(zg.sel(plev=50000, method='nearest').mean())}")
    print(f" - zg250 media: {float(zg.sel(plev=25000, method='nearest').mean())}")
    print(f" - Δz medio (m): {float((zg.sel(plev=25000, method='nearest') - zg.sel(plev=50000, method='nearest')).mean())}")
    print(f" - |u250-u500| media: {float(np.abs((u250 - u500).mean()))}")
    print(f" - |v250-v500| media: {float(np.abs((v250 - v500).mean()))}")
    print(f" - VWS min/max: {float(VWS.min())}/{float(VWS.max())}")

    # Detectar orden de niveles
    plev_vals = ua.plev.values
    ascendente = plev_vals[0] < plev_vals[-1]

    # Definir rango de presiones (Pa) más amplio y con sentido correcto
    if ascendente:
        plev_slice = slice(20000, 60000)
    else:
        plev_slice = slice(60000, 20000)

    # Seleccionar capa
    u_layer = ua.sel(plev=plev_slice)
    v_layer = va.sel(plev=plev_slice)
    zg_layer = zg.sel(plev=plev_slice)
    ta_layer = ta.sel(plev=plev_slice)

    print(f"{modelo}: niveles seleccionados -> {u_layer.plev.values}")

    if u_layer.plev.size < 2:
        print(f" {modelo}: solo {u_layer.plev.size} niveles dentro del rango {plev_slice}, se omite TI1")
        continue

    u_layer = ua.sel(plev=plev_slice)
    v_layer = va.sel(plev=plev_slice)
    zg_layer = zg.sel(plev=plev_slice)
    ta_layer = ta.sel(plev=plev_slice)

    # Calcular el término de deformación (shear tensor)
    DEF_layer = compute_def(u_layer, v_layer)

    p = u_layer["plev"]
    p_vals = p.values.astype(float)
    dp = np.gradient(p_vals)
    w = xr.DataArray(np.abs(dp), dims=["plev"], coords={"plev": p})

    DEF_layer = DEF_layer.where(np.isfinite(DEF_layer))
    DEF_bar = (DEF_layer * w).sum("plev") / w.sum("plev")

    # AÑADE AQUÍ LOS PRINTS DE DIAGNÓSTICO
    print(f"{modelo}: VWS min/max = {float(VWS.min())}/{float(VWS.max())}")
    print(f"{modelo}: DEF_bar min/max = {float(DEF_bar.min())}/{float(DEF_bar.max())}")

    VWS = VWS.where(np.isfinite(VWS))
    DEF_bar = DEF_bar.where(np.isfinite(DEF_bar))
    DEF_bar = DEF_bar.clip(min=0)

    TI1 = (VWS * DEF_bar)
    TI1.attrs["units"] = "s⁻²"
    # TI1 = TI1.clip(min=0, max=20e-7)
    TI1 = TI1.rename("TI1 (s^-2)")

    print(f"{modelo}: TI1 min/max = {float(TI1.min())}/{float(TI1.max())}")

    ti1_min = float(TI1.min())
    ti1_max = float(TI1.max())
    if not np.isfinite(ti1_min) or not np.isfinite(ti1_max) or ti1_max == 0:
        vabs = 1e-6
    else:
        vabs = max(abs(ti1_min), abs(ti1_max))
    vabs = max(vabs, 1e-7)
    norm = mcolors.TwoSlopeNorm(vmin=-vabs, vcenter=0, vmax=vabs)

    TI1_with_year = asignar_invierno(TI1)

    # GUARDAR VALORES DIARIOS DJF PARA PERCENTILES
    ti1_daily_all_models.append(TI1_with_year)
    TI1_djf_yearly = TI1_with_year.groupby("winter_year").mean("time")

    # Media climatológica DJF del TI1 (2030–2100)
    TI1_mean = TI1_djf_yearly.mean("winter_year")
    TI1_mean = TI1_mean.rename("TI1_mean")

    # TI1_mean = TI1_mean.assign_coords(modelo=modelo)
    ti1_mean_models.append(TI1_mean)

    #######################################################################################
    TI1_trend = TI1_djf_yearly.polyfit(dim="winter_year", deg=1)
    TI1_slope = TI1_trend["polyfit_coefficients"].sel(degree=1) * 10
    # TI1_slope = TI1_slope.assign_coords(modelo=modelo)
    ti1_slopes.append(TI1_slope)

# CÁLCULO DEL PORCENTAJE DE DÍAS CON TI₁ > 12×10⁻⁷

UMBRAL_TI1 = 2e-7  # umbral del paper (Jaeger 2007, Ellrod Knapp)

ti1_freq_models = []  # aquí guardaremos las frecuencias (%)

for modelo, TI1 in zip(modelos, ti1_daily_all_models):

    print(f"\n Calculando frecuencia TI₁ > {UMBRAL_TI1:.1e} para {modelo}...")

    # TI₁ ya tiene coordenadas (time, lat, lon[, modelo])
    # Crear máscara booleana (True si TI₁ > umbral)
    exceed = TI1 > UMBRAL_TI1

    # Calcular el porcentaje de días (respecto al total de días válidos)
    #    (equivalente a 100 * sum / count, pero más robusto frente a NaNs)
    freq = exceed.mean(dim="time", skipna=True) * 100

    # Añadir dimensión 'modelo' (solo si no existe)
    if "modelo" not in freq.dims:
        freq = freq.expand_dims(modelo=[modelo])
    else:
        freq = freq.assign_coords(modelo=[modelo])

    freq.name = "TI1_freq"
    freq.attrs["units"] = "%"
    freq.attrs["description"] = f"Porcentaje de días (2030–2100) con TI₁ > {UMBRAL_TI1:.1e} s⁻²"

    # diagnóstico rápido
    print(f"   Máx frecuencia para {modelo}: {float(freq.max()):.3f} %")

    ti1_freq_models.append(freq)

# CÁLCULO DEL PORCENTAJE DE DÍAS CON VWS ≥ 9.7×10⁻³ s⁻¹
UMBRAL_VWS = 9.7e-3  # s^-1

vws_freq_models = []  # aquí guardaremos las frecuencias (%)

for modelo, VWS in zip(modelos, VWS_daily_all_models):

    print(f"\n Calculando frecuencia VWS ≥ {UMBRAL_VWS:.2e} s⁻¹ para {modelo}...")

    # VWS ya tiene coordenadas (time, lat, lon[, modelo])
    # Crear máscara booleana (True si VWS ≥ umbral)
    exceed_vws = VWS >= UMBRAL_VWS

    # Calcular el porcentaje de días (respecto al total de días válidos)
    freq_vws = exceed_vws.mean(dim="time", skipna=True) * 100

    # Añadir dimensión 'modelo' (solo si no existe)
    if "modelo" not in freq_vws.dims:
        freq_vws = freq_vws.expand_dims(modelo=[modelo])
    else:
        freq_vws = freq_vws.assign_coords(modelo=[modelo])

    freq_vws.name = "VWS_freq"
    freq_vws.attrs["units"] = "%"
    freq_vws.attrs["description"] = (
        f"Porcentaje de días (2030–2100) con VWS ≥ {UMBRAL_VWS:.2e} s⁻¹ "
        "(250–500 hPa, DJF)"
    )

    # diagnóstico rápido
    print(f"   Máx frecuencia VWS para {modelo}: {float(freq_vws.max()):.3f} %")

    vws_freq_models.append(freq_vws)

# GUARDAR EL NUEVO ARCHIVO NETCDF
import gc  # Asegurar que está importado

base_out_modelos = os.path.join(BASE_RESULTADOS, escenario, "CAT index", "por_modelo")
os.makedirs(base_out_modelos, exist_ok=True)

if len(ti1_freq_models) > 0:
    print("\n Guardando porcentaje de días con TI₁ > umbral por modelo...")
    ti1_freq_concat = xr.concat(ti1_freq_models, dim="modelo")
    ti1_freq_concat.load()
    ti1_freq_concat.name = "TI1_freq"
    ti1_freq_concat.attrs["units"] = "%"
    ti1_freq_concat.attrs["description"] = (
        f"Frecuencia (2030–2100) de días con TI₁ > {UMBRAL_TI1:.1e} s⁻² "
        "(250–500 hPa, DJF) por modelo"
    )

    freq_path = os.path.join(base_out_modelos, f"TI1_frecuencia_por_modelo_{escenario}_{UMBRAL_TI1}.nc")
    ti1_freq_concat.to_netcdf(freq_path, engine="h5netcdf")
    print(f"✔ Guardado: {freq_path}")

    ti1_freq_concat.close()
    gc.collect()

# GUARDAR PORCENTAJE DE DÍAS CON VWS ≥ UMBRAL

if len(vws_freq_models) > 0:
    print("\n Guardando porcentaje de días con VWS ≥ umbral por modelo...")
    vws_freq_concat = xr.concat(vws_freq_models, dim="modelo")
    vws_freq_concat.load()
    vws_freq_concat.name = "VWS_freq"
    vws_freq_concat.attrs["units"] = "%"
    vws_freq_concat.attrs["description"] = (
        f"Frecuencia (2030–2100) de días con VWS ≥ {UMBRAL_VWS:.2e} s⁻¹ "
        "(250–500 hPa, DJF) por modelo"
    )

    vws_freq_path = os.path.join(
        base_out_modelos,
        f"VWS_frecuencia_por_modelo_{escenario}_{UMBRAL_VWS}.nc"
    )
    vws_freq_concat.to_netcdf(vws_freq_path, engine="h5netcdf")
    print(f"✔ Guardado: {vws_freq_path}")

    vws_freq_concat.close()
    gc.collect()

# GUARDAR RESULTADOS POR MODELO (TI₁ y VWS, 250–500 hPa)

# TI₁: Tendencias
if len(ti1_slopes) > 0:
    print("\n Guardando tendencias de TI₁ por modelo...")
    ti1_tend_concat = xr.concat(ti1_slopes, dim="modelo")
    ti1_tend_concat.load()  # Carga completa en memoria (libera archivos fuente)
    ti1_tend_concat.name = "TI1_tendencia"
    ti1_tend_concat.attrs["units"] = "s^-2 por década"
    ti1_tend_concat.attrs["description"] = "Tendencia de TI₁ (250–500 hPa, DJF 2030–2100) por modelo"
    ti1_tend_path = os.path.join(base_out_modelos, f"TI1_tendencias_por_modelo_{escenario}.nc")
    ti1_tend_concat.to_netcdf(ti1_tend_path, engine="h5netcdf")
    print(f" Guardado: {ti1_tend_path}")
    ti1_tend_concat.close()
    gc.collect()

# TI₁: Medias climatológicas
if len(ti1_mean_models) > 0:
    print("\n Guardando medias climatológicas de TI₁ por modelo...")
    ti1_mean_concat = xr.concat(ti1_mean_models, dim="modelo")
    ti1_mean_concat.load()
    ti1_mean_concat.name = "TI1_media"
    ti1_mean_concat.attrs["units"] = "s^-2"
    ti1_mean_concat.attrs["description"] = "Media climatológica DJF de TI₁ (250–500 hPa, 2030–2100) por modelo"
    ti1_mean_path = os.path.join(base_out_modelos, f"TI1_media_por_modelo_{escenario}.nc")
    ti1_mean_concat.to_netcdf(ti1_mean_path, engine="h5netcdf")
    print(f" Guardado: {ti1_mean_path}")
    ti1_mean_concat.close()
    gc.collect()

# VWS: Tendencias
if len(shear_slopes) > 0:
    print("\n Guardando tendencias de VWS por modelo...")
    VWS_tend_concat = xr.concat(shear_slopes, dim="modelo")
    VWS_tend_concat.load()
    VWS_tend_concat.name = "VWS_tendencia"
    VWS_tend_concat.attrs["units"] = "s^-1 por década"
    VWS_tend_concat.attrs["description"] = "Tendencia de la cizalladura vertical (VWS) en 250–500 hPa (DJF 2030–2100) por modelo"
    VWS_tend_path = os.path.join(base_out_modelos, f"VWS_tendencias_por_modelo_{escenario}.nc")
    VWS_tend_concat.to_netcdf(VWS_tend_path, engine="h5netcdf")
    print(f" Guardado: {VWS_tend_path}")
    VWS_tend_concat.close()
    gc.collect()

# VWS: Medias climatológicas
if len(VWS_mean_models) > 0:
    print("\n Guardando medias climatológicas de VWS por modelo...")
    VWS_mean_concat = xr.concat(VWS_mean_models, dim="modelo")
    VWS_mean_concat.load()
    VWS_mean_concat.name = "VWS_media"
    VWS_mean_concat.attrs["units"] = "s^-1"
    VWS_mean_concat.attrs["description"] = "Media climatológica DJF de la cizalladura vertical (VWS) en 250–500 hPa (2030–2100) por modelo"
    VWS_mean_path = os.path.join(base_out_modelos, f"VWS_media_por_modelo_{escenario}.nc")
    VWS_mean_concat.to_netcdf(VWS_mean_path, engine="h5netcdf")
    print(f" Guardado: {VWS_mean_path}")
    VWS_mean_concat.close()
    gc.collect()

print("\n Comprobando archivos guardados...")
for fname in [
    f"TI1_tendencias_por_modelo_{escenario}.nc",
    f"TI1_media_por_modelo_{escenario}.nc",
    f"VWS_tendencias_por_modelo_{escenario}.nc",
    f"VWS_media_por_modelo_{escenario}.nc"
]:
    fpath = os.path.join(base_out_modelos, fname)
    if os.path.exists(fpath):
        ds = xr.open_dataset(fpath)
        print(f" {fname}: {list(ds.dims.keys())} | modelos = {len(ds.modelo)}")
        ds.close()
    else:
        print(f" No se encontró {fname}")